In [10]:
import pickle
with open("17_mnist.pkl", "br") as fh:
    data = pickle.load(fh)

train_imgs = data[0]
test_imgs = data[1]
train_labels = data[2]
test_labels = data[3]



In [11]:
import numpy as np
for i in range(len(train_labels)):
    if train_labels[i]==7:
        train_labels[i]=np.array([0.])

In [12]:
train_x = torch.FloatTensor(train_imgs)
train_y = torch.FloatTensor(train_labels)
#train_y = train_y.squeeze(1)

In [13]:
print(train_y.size())

torch.Size([13007, 1])


In [14]:
class LinearDataset(Dataset):
    def __init__(self, X, y):
        assert X.size()[0] == y.size()[0]
        self.X = X
        self.y = y
    
    def __len__(self):
        return self.X.size()[0]
    
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

In [15]:
import os
import time
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from numpy import linalg as LA
import numpy as np
import math

device='cpu'

class LR(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, num_class=1):
        super(LR, self).__init__()
        self.fc1 = nn.Linear(input_size, num_class)
        self.act=nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        return (x)

In [16]:
train_loader = torch.utils.data.DataLoader(LinearDataset(train_x, train_y), batch_size=len(train_imgs))

In [17]:
model = LR().to(device).double()

In [83]:
state_dict = model.state_dict()
state_dict['fc1.weight'] = torch.ones([1,784])
state_dict['fc1.bias'] = torch.Tensor([-0.001])
model.load_state_dict(state_dict)

<All keys matched successfully>

In [84]:
for name, param in model.named_parameters():
    if param.requires_grad:
        #print(name, param.data)
        break
     

In [85]:
optimizer = optim.Adadelta(model.parameters(), lr=0.1)
# define epsilon for one point for now
epsilon = 1/len(train_imgs)
# alpha may indicate the scaling factor?
alpha = 0


def autograd(outputs, inputs, create_graph=False):
    """Compute gradient of outputs w.r.t. inputs, assuming outputs is a scalar."""
    #inputs = tuple(inputs)
    grads = torch.autograd.grad(outputs, inputs, create_graph=create_graph, allow_unused=True)
    return [xx if xx is not None else yy.new_zeros(yy.size()) for xx, yy in zip(grads, inputs)]

from scipy.special import lambertw
import math
lambert_w = lambertw(1/math.e)

# load gmu to initialize the poisoned samples
g1 = torch.load('gmu.pt')

loss_all = []
def attack(epoch,lr):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).double(), target.to(device).double()
        data.requires_grad=True
        if epoch==0:
            # initialize poisoned data with 1/epsilon g(mu)
            #data_p = Variable(data[:(int(epsilon*len(data)))])
            data_p = Variable(g1.repeat(epsilon))
            target_p = Variable(target[:(int(epsilon*len(target)))])
            torch.save(target_p,'poisoned_models/lr/target_p_{}.pt'.format(epsilon))
        else:
            data_p = torch.load('poisoned_models/lr/data_p_{}.pt'.format(epsilon))
            target_p = torch.load('poisoned_models/lr/target_p_{}.pt'.format(epsilon))
        data_p.requires_grad=True
        # initialize f function
        criterion = nn.BCELoss(reduction='sum')
        
        # calculate gradient of w on clean sample
        output_c = model(data.view(data.size(0), -1))
        #output_c = model(data)
        loss_c = criterion(output_c,target)
        # wrt to w here
        grad_c= autograd(loss_c,tuple(model.parameters()),create_graph=True)
        g1 = grad_c[0]
        
        # calculate gradient of w on poisoned sample
        output_p = model(data_p.view(data_p.size(0), -1))
        loss_p = criterion(output_p,target_p)
        grad_p= autograd(loss_p,tuple(model.parameters()),create_graph=True)
        g2 = grad_p[0]
        
        # calculate the true loss: |g_c + g_p|_{inf}
        
        grad_sum = g1+g2

        
        loss = torch.norm(grad_sum,2)
        loss_all.append(loss.detach().cpu().numpy())
        if loss < 1:
            break
            
        update = autograd(loss,data_p,create_graph=True)
        
        data_t = data_p - lr * update[0]

        torch.save(data_t, 'poisoned_models/lr/data_p_{}.pt'.format(epsilon))
        
        
        print("epoch:{},loss:{},lr:{}".format(epoch, loss,lr))

        
        
print("==> start gradient canceling attack with given target parameters")
print("==> model will be saved in poisoned_models")
epochs=20
lr=1
for epoch in range(epochs):
    attack(epoch,lr)
    break
    

==> start gradient canceling attack with given target parameters
==> model will be saved in poisoned_models
tensor(626500., dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward>)
the necessary size of epsilon_d:(28.765904788638146+0j)


In [86]:
a = torch.Tensor([1])

In [89]:
print(a.repeat(5))

tensor([1., 1., 1., 1., 1.])
